In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/google-cloud-ncaa-march-madness-2020-division-1-mens-tournament/MEvents2016.csv
/kaggle/input/google-cloud-ncaa-march-madness-2020-division-1-mens-tournament/MEvents2019.csv
/kaggle/input/google-cloud-ncaa-march-madness-2020-division-1-mens-tournament/MSampleSubmissionStage1_2020.csv
/kaggle/input/google-cloud-ncaa-march-madness-2020-division-1-mens-tournament/MEvents2017.csv
/kaggle/input/google-cloud-ncaa-march-madness-2020-division-1-mens-tournament/MPlayers.csv
/kaggle/input/google-cloud-ncaa-march-madness-2020-division-1-mens-tournament/MEvents2018.csv
/kaggle/input/google-cloud-ncaa-march-madness-2020-division-1-mens-tournament/MEvents2015.csv
/kaggle/input/google-cloud-ncaa-march-madness-2020-division-1-mens-tournament/MDataFiles_Stage1/Cities.csv
/kaggle/input/google-cloud-ncaa-march-madness-2020-division-1-mens-tournament/MDataFiles_Stage1/MSecondaryTourneyTeams.csv
/kaggle/input/google-cloud-ncaa-march-madness-2020-division-1-mens-tournament/MDataFiles_Stage1/MN

In [2]:
import pandas as pd
import numpy as np
import os
import sys

import re
import random
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
import lightgbm as lgb
import xgboost as xgb
from xgboost import XGBClassifier
import gc
import glob

%matplotlib inline

In [3]:
df = {}
for i in glob.glob('/kaggle/input/google-cloud-ncaa-march-madness-2020-division-1-mens-tournament/MDataFiles_Stage1/*'):
    name = i.split('/')[-1].split('.')[0]
    if name != 'MTeamSpellings':
        df[name] = pd.read_csv(i)
    else:
        df[name] = pd.read_csv(i, encoding='cp1252')

In [4]:
df.keys()

dict_keys(['Cities', 'MSecondaryTourneyTeams', 'MNCAATourneyCompactResults', 'MRegularSeasonCompactResults', 'MTeamConferences', 'MNCAATourneyDetailedResults', 'MTeams', 'MNCAATourneySlots', 'MTeamCoaches', 'MNCAATourneySeedRoundSlots', 'Conferences', 'MConferenceTourneyGames', 'MTeamSpellings', 'MNCAATourneySeeds', 'MRegularSeasonDetailedResults', 'MGameCities', 'MSecondaryTourneyCompactResults', 'MMasseyOrdinals', 'MSeasons'])

In [5]:
regular=df['MRegularSeasonDetailedResults']
regular=regular[['Season','WTeamID','LTeamID','WScore', 'LScore','WLoc','WFGM', 'WFGA',
       'WFGM3', 'WFGA3', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3']]
regular

,Season,WTeamID,LTeamID,WScore,LScore,WLoc,WFGM,WFGA,WFGM3,WFGA3,LFGM,LFGA,LFGM3,LFGA3
0,2003,1104,1328,68,62,N,27,58,3,14,22,53,2,10
1,2003,1272,1393,70,63,N,26,62,8,20,24,67,6,24
2,2003,1266,1437,73,61,N,24,58,8,18,22,73,3,26
3,2003,1296,1457,56,50,N,18,38,3,9,18,49,6,22
4,2003,1400,1208,77,71,N,30,61,6,14,24,62,6,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87499,2019,1153,1222,69,57,N,22,50,6,17,19,62,8,33
87500,2019,1209,1426,73,64,N,20,50,8,22,23,64,7,33
87501,2019,1277,1276,65,60,N,22,55,9,23,21,51,8,25
87502,2019,1387,1382,55,53,N,22,59,4,22,19,56,7,19


In [6]:
tourney=df['MNCAATourneyDetailedResults']
tourney=tourney[['Season','WTeamID','LTeamID','WScore', 'LScore','WLoc','WFGM', 'WFGA',
       'WFGM3', 'WFGA3', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3']]
tourney

,Season,WTeamID,LTeamID,WScore,LScore,WLoc,WFGM,WFGA,WFGM3,WFGA3,LFGM,LFGA,LFGM3,LFGA3
0,2003,1421,1411,92,84,N,32,69,11,29,29,67,12,31
1,2003,1112,1436,80,51,N,31,66,7,23,20,64,4,16
2,2003,1113,1272,84,71,N,31,59,6,14,25,69,7,28
3,2003,1141,1166,79,73,N,29,53,3,7,27,60,7,17
4,2003,1143,1301,76,74,N,27,64,7,20,25,56,9,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1110,2019,1120,1246,77,71,N,26,65,7,23,27,61,5,21
1111,2019,1277,1181,68,67,N,30,70,6,19,26,57,7,21
1112,2019,1403,1277,61,51,N,22,51,9,23,15,47,7,24
1113,2019,1438,1120,63,62,N,25,51,7,19,21,55,9,31


In [7]:
rw=regular[['Season','WTeamID','WFGM','WFGA', 'WFGM3', 'WFGA3']]
rw=rw.groupby(['Season','WTeamID']).mean()

tw=regular[['Season','WTeamID','WFGM','WFGA', 'WFGM3', 'WFGA3']]
tw=tw.groupby(['Season','WTeamID']).mean()

rtw=pd.concat((rw,tw))
rtw=rtw.groupby(['Season','WTeamID']).mean()
rtw

WFGM       WFGA      WFGM3      WFGA3
Season WTeamID                                            
2003   1102     22.583333  40.000000  10.000000  21.583333
       1103     30.000000  55.384615   5.461538  14.384615
       1104     25.823529  58.352941   7.058824  20.823529
       1105     25.571429  61.857143   9.142857  22.428571
       1106     24.769231  53.846154   5.846154  15.923077
...                   ...        ...        ...        ...
2019   1462     27.666667  55.388889   7.555556  21.000000
       1463     31.000000  59.857143   8.000000  20.000000
       1464     28.000000  63.400000  10.100000  27.600000
       1465     27.416667  57.500000   9.583333  24.333333
       1466     24.857143  59.857143   8.000000  23.857143

[5829 rows x 4 columns]

In [8]:
rl=regular[['Season','LTeamID','LFGM','LFGA', 'LFGM3', 'LFGA3']]
rl=rl.groupby(['Season','LTeamID']).mean()

tl=regular[['Season','LTeamID','LFGM','LFGA', 'LFGM3', 'LFGA3']]
tl=tl.groupby(['Season','LTeamID']).mean()

rtl=pd.concat((rl,tl))
rtl=rtl.groupby(['Season','LTeamID']).mean()
rtl

LFGM       LFGA     LFGM3      LFGA3
Season LTeamID                                           
2003   1102     16.562500  39.625000  6.187500  20.250000
       1103     24.500000  56.285714  5.428571  17.642857
       1104     21.272727  55.363636  5.272727  18.363636
       1105     23.947368  61.526316  7.000000  20.157895
       1106     22.266667  56.533333  6.333333  19.133333
...                   ...        ...       ...        ...
2019   1462     24.266667  56.400000  6.333333  21.066667
       1463     26.285714  60.857143  7.142857  23.285714
       1464     26.250000  63.750000  9.300000  28.200000
       1465     24.857143  60.357143  8.142857  26.000000
       1466     22.318182  59.590909  7.000000  24.500000

[5832 rows x 4 columns]

In [9]:
secondary=df['MSecondaryTourneyCompactResults']
secondary=secondary[['Season','WTeamID','LTeamID','WScore', 'LScore','WLoc']]


secondary=pd.merge(secondary,rtw,how='left',left_on=['Season','WTeamID'],right_on=['Season','WTeamID'])
secondary=pd.merge(secondary,rtl,how='left',left_on=['Season','LTeamID'],right_on=['Season','LTeamID'])
secondary

,Season,WTeamID,LTeamID,WScore,LScore,WLoc,WFGM,WFGA,WFGM3,WFGA3,LFGM,LFGA,LFGM3,LFGA3
0,1985,1151,1155,67,65,H,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1985,1153,1245,77,61,H,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1985,1201,1365,79,76,H,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1985,1231,1139,79,57,H,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1985,1249,1222,78,71,H,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1619,2019,1453,1411,87,86,H,29.933333,62.733333,8.066667,21.733333,25.538462,65.923077,6.461538,25.153846
1620,2019,1177,1378,100,96,H,29.066667,58.866667,7.066667,19.600000,22.615385,56.615385,6.769231,23.538462
1621,2019,1267,1453,90,70,H,30.157895,62.789474,10.736842,27.894737,25.750000,62.187500,7.500000,24.000000
1622,2019,1400,1252,81,66,N,26.437500,56.562500,9.625000,25.312500,24.000000,61.142857,6.428571,20.857143


In [10]:
secondary.isna().sum()

Season       0
WTeamID      0
LTeamID      0
WScore       0
LScore       0
WLoc         0
WFGM       584
WFGA       584
WFGM3      584
WFGA3      584
LFGM       584
LFGA       584
LFGM3      584
LFGA3      584
dtype: int64

In [11]:
s=secondary[(secondary['Season']<2003)]
s

,Season,WTeamID,LTeamID,WScore,LScore,WLoc,WFGM,WFGA,WFGM3,WFGA3,LFGM,LFGA,LFGM3,LFGA3
0,1985,1151,1155,67,65,H,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1985,1153,1245,77,61,H,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1985,1201,1365,79,76,H,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1985,1231,1139,79,57,H,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1985,1249,1222,78,71,H,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
579,2002,1272,1399,79,73,H,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
580,2002,1272,1396,78,77,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
581,2002,1376,1393,66,59,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
582,2002,1272,1376,72,62,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
secondary=secondary.dropna()
secondary

,Season,WTeamID,LTeamID,WScore,LScore,WLoc,WFGM,WFGA,WFGM3,WFGA3,LFGM,LFGA,LFGM3,LFGA3
584,2003,1234,1434,62,60,H,24.533333,53.200000,4.800000,16.066667,20.500000,53.800000,4.000000,17.300000
585,2003,1130,1193,90,78,A,29.444444,58.166667,6.500000,16.555556,22.818182,54.090909,4.090909,13.181818
586,2003,1207,1397,70,60,A,27.333333,59.933333,4.933333,13.333333,23.727273,55.090909,6.363636,18.272727
587,2003,1314,1177,83,72,H,26.411765,57.411765,7.705882,21.000000,21.833333,52.416667,3.250000,12.666667
588,2003,1373,1437,74,59,H,25.684211,57.157895,6.000000,16.210526,23.333333,59.133333,5.800000,19.533333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1619,2019,1453,1411,87,86,H,29.933333,62.733333,8.066667,21.733333,25.538462,65.923077,6.461538,25.153846
1620,2019,1177,1378,100,96,H,29.066667,58.866667,7.066667,19.600000,22.615385,56.615385,6.769231,23.538462
1621,2019,1267,1453,90,70,H,30.157895,62.789474,10.736842,27.894737,25.750000,62.187500,7.500000,24.000000
1622,2019,1400,1252,81,66,N,26.437500,56.562500,9.625000,25.312500,24.000000,61.142857,6.428571,20.857143


In [13]:
all_match=pd.concat((regular,tourney,secondary))
all_match

,Season,WTeamID,LTeamID,WScore,LScore,WLoc,WFGM,WFGA,WFGM3,WFGA3,LFGM,LFGA,LFGM3,LFGA3
0,2003,1104,1328,68,62,N,27.000000,58.000000,3.000000,14.000000,22.000000,53.000000,2.000000,10.000000
1,2003,1272,1393,70,63,N,26.000000,62.000000,8.000000,20.000000,24.000000,67.000000,6.000000,24.000000
2,2003,1266,1437,73,61,N,24.000000,58.000000,8.000000,18.000000,22.000000,73.000000,3.000000,26.000000
3,2003,1296,1457,56,50,N,18.000000,38.000000,3.000000,9.000000,18.000000,49.000000,6.000000,22.000000
4,2003,1400,1208,77,71,N,30.000000,61.000000,6.000000,14.000000,24.000000,62.000000,6.000000,16.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1619,2019,1453,1411,87,86,H,29.933333,62.733333,8.066667,21.733333,25.538462,65.923077,6.461538,25.153846
1620,2019,1177,1378,100,96,H,29.066667,58.866667,7.066667,19.600000,22.615385,56.615385,6.769231,23.538462
1621,2019,1267,1453,90,70,H,30.157895,62.789474,10.736842,27.894737,25.750000,62.187500,7.500000,24.000000
1622,2019,1400,1252,81,66,N,26.437500,56.562500,9.625000,25.312500,24.000000,61.142857,6.428571,20.857143


In [14]:
all_match.isna().sum()

Season     0
WTeamID    0
LTeamID    0
WScore     0
LScore     0
WLoc       0
WFGM       0
WFGA       0
WFGM3      0
WFGA3      0
LFGM       0
LFGA       0
LFGM3      0
LFGA3      0
dtype: int64

In [15]:

all_match.groupby('WLoc').size()


WLoc
A    27053
H    52511
N    10095
dtype: int64

In [16]:
seeds=df['MNCAATourneySeeds']
seeds

,Season,Seed,TeamID
0,1985,W01,1207
1,1985,W02,1210
2,1985,W03,1228
3,1985,W04,1260
4,1985,W05,1374
...,...,...,...
2281,2019,Z12,1332
2282,2019,Z13,1414
2283,2019,Z14,1330
2284,2019,Z15,1159


In [17]:
seeds['Seed'] = seeds['Seed'].apply(lambda x: int(x[1:3]))
seeds

,Season,Seed,TeamID
0,1985,1,1207
1,1985,2,1210
2,1985,3,1228
3,1985,4,1260
4,1985,5,1374
...,...,...,...
2281,2019,12,1332
2282,2019,13,1414
2283,2019,14,1330
2284,2019,15,1159


In [18]:
data=pd.merge(all_match,seeds,how='left',left_on=['Season','WTeamID'],right_on=['Season','TeamID'])
data.drop(['TeamID'],axis=1,inplace=True)
data.rename(columns={'Seed':'WSeed'},inplace=True)

data=pd.merge(data,seeds,how='left',left_on=['Season','LTeamID'],right_on=['Season','TeamID'])
data.drop(['TeamID'],axis=1,inplace=True)
data.rename(columns={'Seed':'LSeed'},inplace=True)

data=data[['Season', 'WTeamID', 'LTeamID', 'WSeed','LSeed', 'WScore', 'LScore', 'WLoc', 
           'WFGM','WFGA', 'WFGM3', 'WFGA3', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3']]
data

,Season,WTeamID,LTeamID,WSeed,LSeed,WScore,LScore,WLoc,WFGM,WFGA,WFGM3,WFGA3,LFGM,LFGA,LFGM3,LFGA3
0,2003,1104,1328,10.0,1.0,68,62,N,27.000000,58.000000,3.000000,14.000000,22.000000,53.000000,2.000000,10.000000
1,2003,1272,1393,7.0,3.0,70,63,N,26.000000,62.000000,8.000000,20.000000,24.000000,67.000000,6.000000,24.000000
2,2003,1266,1437,3.0,NaN,73,61,N,24.000000,58.000000,8.000000,18.000000,22.000000,73.000000,3.000000,26.000000
3,2003,1296,1457,NaN,NaN,56,50,N,18.000000,38.000000,3.000000,9.000000,18.000000,49.000000,6.000000,22.000000
4,2003,1400,1208,1.0,NaN,77,71,N,30.000000,61.000000,6.000000,14.000000,24.000000,62.000000,6.000000,16.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89654,2019,1453,1411,NaN,NaN,87,86,H,29.933333,62.733333,8.066667,21.733333,25.538462,65.923077,6.461538,25.153846
89655,2019,1177,1378,NaN,NaN,100,96,H,29.066667,58.866667,7.066667,19.600000,22.615385,56.615385,6.769231,23.538462
89656,2019,1267,1453,NaN,NaN,90,70,H,30.157895,62.789474,10.736842,27.894737,25.750000,62.187500,7.500000,24.000000
89657,2019,1400,1252,NaN,NaN,81,66,N,26.437500,56.562500,9.625000,25.312500,24.000000,61.142857,6.428571,20.857143


In [19]:
data.isna().sum()
data=data.dropna()
data=data.reset_index(drop=True)

In [20]:
data.rename(columns={'WTeamID':'TeamA','LTeamID':'TeamB'},inplace=True)
data.rename(columns={'WScore':'AScore','LScore':'BScore'},inplace=True)
data.rename(columns={'WSeed':'ASeed','LSeed':'BSeed','WLoc':'Loc'},inplace=True)

data

,Season,TeamA,TeamB,ASeed,BSeed,AScore,BScore,Loc,WFGM,WFGA,WFGM3,WFGA3,LFGM,LFGA,LFGM3,LFGA3
0,2003,1104,1328,10.0,1.0,68,62,N,27.0,58.0,3.0,14.0,22.0,53.0,2.0,10.0
1,2003,1272,1393,7.0,3.0,70,63,N,26.0,62.0,8.0,20.0,24.0,67.0,6.0,24.0
2,2003,1323,1237,5.0,16.0,89,45,H,34.0,62.0,8.0,16.0,18.0,74.0,4.0,15.0
3,2003,1242,1221,2.0,14.0,81,57,H,28.0,53.0,3.0,5.0,18.0,52.0,7.0,21.0
4,2003,1390,1462,4.0,3.0,63,62,H,23.0,57.0,6.0,15.0,18.0,45.0,6.0,20.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6067,2019,1120,1246,5.0,2.0,77,71,N,26.0,65.0,7.0,23.0,27.0,61.0,5.0,21.0
6068,2019,1277,1181,2.0,1.0,68,67,N,30.0,70.0,6.0,19.0,26.0,57.0,7.0,21.0
6069,2019,1403,1277,3.0,2.0,61,51,N,22.0,51.0,9.0,23.0,15.0,47.0,7.0,24.0
6070,2019,1438,1120,1.0,5.0,63,62,N,25.0,51.0,7.0,19.0,21.0,55.0,9.0,31.0


In [21]:
win=data.copy()
lose=data.copy()

lose['TeamA']=win['TeamB']
lose['TeamB']=win['TeamA']

lose['ASeed']=win['BSeed']
lose['BSeed']=win['ASeed']

lose['AScore']=win['BScore']
lose['BScore']=win['AScore']

lose['Loc']=lose['Loc'].apply(lambda x: 'H' if x=='N' else 'N' if x=='H' else 'A')

win['result']=1
lose['result']=0

total=pd.concat((win,lose))
total['Loc']=total['Loc'].apply(lambda x: '1' if x=='H' else '0' if x=='N' else '-1')

total['Seed_diff']=total['ASeed']-total['BSeed']
total['Score_diff']=total['AScore']-total['BScore']


train=total[['Season', 'TeamA', 'TeamB', 'ASeed', 'BSeed', 'AScore', 'BScore', 'Loc',
       'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3', 'Seed_diff', 'Score_diff',
       'result']]
train


,Season,TeamA,TeamB,ASeed,BSeed,AScore,BScore,Loc,WFGM,WFGA,WFGM3,WFGA3,LFGM,LFGA,LFGM3,LFGA3,Seed_diff,Score_diff,result
0,2003,1104,1328,10.0,1.0,68,62,0,27.0,58.0,3.0,14.0,22.0,53.0,2.0,10.0,9.0,6,1
1,2003,1272,1393,7.0,3.0,70,63,0,26.0,62.0,8.0,20.0,24.0,67.0,6.0,24.0,4.0,7,1
2,2003,1323,1237,5.0,16.0,89,45,1,34.0,62.0,8.0,16.0,18.0,74.0,4.0,15.0,-11.0,44,1
3,2003,1242,1221,2.0,14.0,81,57,1,28.0,53.0,3.0,5.0,18.0,52.0,7.0,21.0,-12.0,24,1
4,2003,1390,1462,4.0,3.0,63,62,1,23.0,57.0,6.0,15.0,18.0,45.0,6.0,20.0,1.0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6067,2019,1246,1120,2.0,5.0,71,77,1,26.0,65.0,7.0,23.0,27.0,61.0,5.0,21.0,-3.0,-6,0
6068,2019,1181,1277,1.0,2.0,67,68,1,30.0,70.0,6.0,19.0,26.0,57.0,7.0,21.0,-1.0,-1,0
6069,2019,1277,1403,2.0,3.0,51,61,1,22.0,51.0,9.0,23.0,15.0,47.0,7.0,24.0,-1.0,-10,0
6070,2019,1120,1438,5.0,1.0,62,63,1,25.0,51.0,7.0,19.0,21.0,55.0,9.0,31.0,4.0,-1,0


test data

In [22]:
test=pd.read_csv('/kaggle/input/google-cloud-ncaa-march-madness-2020-division-1-mens-tournament/MSampleSubmissionStage1_2020.csv')
test

,ID,Pred
0,2015_1107_1112,0.5
1,2015_1107_1116,0.5
2,2015_1107_1124,0.5
3,2015_1107_1125,0.5
4,2015_1107_1129,0.5
...,...,...
11385,2019_1449_1459,0.5
11386,2019_1449_1463,0.5
11387,2019_1458_1459,0.5
11388,2019_1458_1463,0.5


In [23]:
test[['Season','A','B']]=test['ID'].str.split('_',expand=True)
test['Season']=test['Season'].astype(int)
test['A']=test['A'].astype(int)
test['B']=test['B'].astype(int)
test.drop(['ID','Pred'],axis=1,inplace=True)
test

,Season,A,B
0,2015,1107,1112
1,2015,1107,1116
2,2015,1107,1124
3,2015,1107,1125
4,2015,1107,1129
...,...,...,...
11385,2019,1449,1459
11386,2019,1449,1463
11387,2019,1458,1459
11388,2019,1458,1463


In [24]:
test=pd.merge(test,seeds,how='left',left_on=['Season','A'],right_on=['Season','TeamID'])
test

,Season,A,B,Seed,TeamID
0,2015,1107,1112,14,1107
1,2015,1107,1116,14,1107
2,2015,1107,1124,14,1107
3,2015,1107,1125,14,1107
4,2015,1107,1129,14,1107
...,...,...,...,...,...
11385,2019,1449,1459,9,1449
11386,2019,1449,1463,9,1449
11387,2019,1458,1459,5,1458
11388,2019,1458,1463,5,1458
